# 📝 Documentation Copilot - Geração de Documentação de API

Este notebook demonstra como usar o **Documentation Copilot** para gerar documentação automaticamente.

## Funcionalidades

- 📚 Gerar docstrings (Google, NumPy, Sphinx)
- 📖 Criar documentação de API
- 📋 Gerar README.md
- 💬 Adicionar comentários explicativos
- 📄 Documentar endpoints REST


## 1. Setup Inicial


In [1]:
# Configuração do ambiente
import sys
sys.path.insert(0, '..')

from src.utils import setup_logging
setup_logging(level="INFO")

print("✅ Setup concluído!")


✅ Setup concluído!


In [2]:
# Importar o Documentation Copilot
from src.copilots import DocumentationCopilot

# Inicializar o copiloto
doc_copilot = DocumentationCopilot()

print(f"📝 Documentation Copilot inicializado!")
print(f"   Modelo: {doc_copilot.model_config.name}")
print(f"   Provider: {doc_copilot.model_config.provider}")


[12/11/25 16:11:05] INFO     2025-12-11 16:11:05,328 - src.agents.base - INFO - Copiloto 'Documentation Writer'    
                             inicializado com openai/gpt-4o

📝 Documentation Copilot inicializado!
   Modelo: gpt-4o
   Provider: openai


## 2. Gerar Docstrings

Vamos gerar docstrings para funções sem documentação.


In [3]:
# Código sem documentação
codigo_sem_docs = '''
def calcular_juros_compostos(principal, taxa, periodos, capitalizacao="mensal"):
    if capitalizacao == "mensal":
        n = 12
    elif capitalizacao == "trimestral":
        n = 4
    elif capitalizacao == "semestral":
        n = 2
    else:
        n = 1
    
    montante = principal * (1 + taxa / n) ** (n * periodos)
    juros = montante - principal
    
    return {
        "principal": principal,
        "montante": round(montante, 2),
        "juros": round(juros, 2),
        "taxa_efetiva": round((montante / principal - 1) * 100, 2)
    }

def validar_cpf(cpf):
    cpf = ''.join(filter(str.isdigit, cpf))
    
    if len(cpf) != 11:
        return False
    
    if cpf == cpf[0] * 11:
        return False
    
    soma = sum(int(cpf[i]) * (10 - i) for i in range(9))
    resto = soma % 11
    digito1 = 0 if resto < 2 else 11 - resto
    
    if int(cpf[9]) != digito1:
        return False
    
    soma = sum(int(cpf[i]) * (11 - i) for i in range(10))
    resto = soma % 11
    digito2 = 0 if resto < 2 else 11 - resto
    
    return int(cpf[10]) == digito2
'''

print("📝 Código sem documentação:")
print(codigo_sem_docs)


📝 Código sem documentação:

def calcular_juros_compostos(principal, taxa, periodos, capitalizacao="mensal"):
    if capitalizacao == "mensal":
        n = 12
    elif capitalizacao == "trimestral":
        n = 4
    elif capitalizacao == "semestral":
        n = 2
    else:
        n = 1

    montante = principal * (1 + taxa / n) ** (n * periodos)
    juros = montante - principal

    return {
        "principal": principal,
        "montante": round(montante, 2),
        "juros": round(juros, 2),
        "taxa_efetiva": round((montante / principal - 1) * 100, 2)
    }

def validar_cpf(cpf):
    cpf = ''.join(filter(str.isdigit, cpf))

    if len(cpf) != 11:
        return False

    if cpf == cpf[0] * 11:
        return False

    soma = sum(int(cpf[i]) * (10 - i) for i in range(9))
    resto = soma % 11
    digito1 = 0 if resto < 2 else 11 - resto

    if int(cpf[9]) != digito1:
        return False

    soma = sum(int(cpf[i]) * (11 - i) for i in range(10))
    resto = soma % 11
    

In [4]:
# Importar o enum de estilos
from src.copilots.documentation import DocstringStyle

# Gerar docstrings estilo Google
print("📚 Gerando docstrings (estilo Google)...\n")

resultado = doc_copilot.generate_docstring(
    code=codigo_sem_docs,
    language="python",
    style=DocstringStyle.GOOGLE  # Usar o enum ao invés de string
)

if resultado.success:
    print("✅ Docstrings geradas:\n")
    print(resultado.content)
else:
    print(f"❌ Erro: {resultado.content}")


📚 Gerando docstrings (estilo Google)...

✅ Docstrings geradas:

```python
def calcular_juros_compostos(principal, taxa, periodos, capitalizacao="mensal"):
    """Calcula o montante e os juros compostos para um investimento.

    Este método calcula o montante final e os juros acumulados de um investimento
    baseado no valor principal, taxa de juros, número de períodos e tipo de capitalização.

    Args:
        principal (float): O valor inicial do investimento.
        taxa (float): A taxa de juros anual, em decimal (por exemplo, 0.05 para 5%).
        periodos (int): O número de períodos de investimento.
        capitalizacao (str, opcional): A frequência de capitalização dos juros. 
            Pode ser "mensal", "trimestral", "semestral" ou "anual". 
            Padrão é "mensal".

    Returns:
        dict: Um dicionário contendo:
            - "principal" (float): O valor inicial do investimento.
            - "montante" (float): O montante total após a aplicação dos juros.
   

## 3. Documentar API REST (FastAPI)

Gerar documentação completa para endpoints de uma API.


In [5]:
# API FastAPI sem documentação
codigo_api = '''
from fastapi import FastAPI, HTTPException, Query, Path
from pydantic import BaseModel, Field
from typing import List, Optional
from datetime import datetime

app = FastAPI()

class Produto(BaseModel):
    id: int
    nome: str
    descricao: Optional[str] = None
    preco: float
    estoque: int
    categoria: str
    ativo: bool = True
    criado_em: datetime = Field(default_factory=datetime.now)

class ProdutoCreate(BaseModel):
    nome: str
    descricao: Optional[str] = None
    preco: float
    estoque: int
    categoria: str

class ProdutoUpdate(BaseModel):
    nome: Optional[str] = None
    descricao: Optional[str] = None
    preco: Optional[float] = None
    estoque: Optional[int] = None
    categoria: Optional[str] = None
    ativo: Optional[bool] = None

produtos_db = {}

@app.get("/produtos")
def listar_produtos(
    categoria: Optional[str] = None,
    ativo: bool = True,
    min_preco: Optional[float] = None,
    max_preco: Optional[float] = None,
    skip: int = 0,
    limit: int = 100
):
    resultado = list(produtos_db.values())
    
    if categoria:
        resultado = [p for p in resultado if p.categoria == categoria]
    if ativo is not None:
        resultado = [p for p in resultado if p.ativo == ativo]
    if min_preco:
        resultado = [p for p in resultado if p.preco >= min_preco]
    if max_preco:
        resultado = [p for p in resultado if p.preco <= max_preco]
    
    return resultado[skip:skip + limit]

@app.get("/produtos/{produto_id}")
def obter_produto(produto_id: int):
    if produto_id not in produtos_db:
        raise HTTPException(status_code=404, detail="Produto não encontrado")
    return produtos_db[produto_id]

@app.post("/produtos", status_code=201)
def criar_produto(produto: ProdutoCreate):
    novo_id = max(produtos_db.keys(), default=0) + 1
    novo_produto = Produto(id=novo_id, **produto.dict())
    produtos_db[novo_id] = novo_produto
    return novo_produto

@app.put("/produtos/{produto_id}")
def atualizar_produto(produto_id: int, produto: ProdutoUpdate):
    if produto_id not in produtos_db:
        raise HTTPException(status_code=404, detail="Produto não encontrado")
    
    produto_atual = produtos_db[produto_id]
    dados_atualizados = produto.dict(exclude_unset=True)
    
    for campo, valor in dados_atualizados.items():
        setattr(produto_atual, campo, valor)
    
    return produto_atual

@app.delete("/produtos/{produto_id}", status_code=204)
def deletar_produto(produto_id: int):
    if produto_id not in produtos_db:
        raise HTTPException(status_code=404, detail="Produto não encontrado")
    del produtos_db[produto_id]

@app.get("/produtos/busca/")
def buscar_produtos(q: str = Query(..., min_length=2)):
    resultado = [
        p for p in produtos_db.values()
        if q.lower() in p.nome.lower() or (p.descricao and q.lower() in p.descricao.lower())
    ]
    return resultado
'''

print("📝 API FastAPI:")
print(codigo_api[:1500] + "\n... (código continua)")


📝 API FastAPI:

from fastapi import FastAPI, HTTPException, Query, Path
from pydantic import BaseModel, Field
from typing import List, Optional
from datetime import datetime

app = FastAPI()

class Produto(BaseModel):
    id: int
    nome: str
    descricao: Optional[str] = None
    preco: float
    estoque: int
    categoria: str
    ativo: bool = True
    criado_em: datetime = Field(default_factory=datetime.now)

class ProdutoCreate(BaseModel):
    nome: str
    descricao: Optional[str] = None
    preco: float
    estoque: int
    categoria: str

class ProdutoUpdate(BaseModel):
    nome: Optional[str] = None
    descricao: Optional[str] = None
    preco: Optional[float] = None
    estoque: Optional[int] = None
    categoria: Optional[str] = None
    ativo: Optional[bool] = None

produtos_db = {}

@app.get("/produtos")
def listar_produtos(
    categoria: Optional[str] = None,
    ativo: bool = True,
    min_preco: Optional[float] = None,
    max_preco: Optional[float] = None,
    skip

In [6]:
# Gerar documentação de API
print("📖 Gerando documentação de API...\n")

resultado = doc_copilot.generate_docstring(
    code=codigo_api,
    language="python",
    style=DocstringStyle.GOOGLE  # Usar o enum
)

if resultado.success:
    print("✅ Documentação de API gerada:\n")
    print(resultado.content)
else:
    print(f"❌ Erro: {resultado.content}")


📖 Gerando documentação de API...

✅ Documentação de API gerada:

```python
def listar_produtos(
    categoria: Optional[str] = None,
    ativo: bool = True,
    min_preco: Optional[float] = None,
    max_preco: Optional[float] = None,
    skip: int = 0,
    limit: int = 100
):
    """Lista produtos com base em filtros opcionais e paginação.

    Args:
        categoria (Optional[str]): Categoria do produto para filtrar. Se None, não filtra por categoria.
        ativo (bool): Indica se deve filtrar apenas produtos ativos. Padrão é True.
        min_preco (Optional[float]): Preço mínimo do produto para filtrar. Se None, não filtra por preço mínimo.
        max_preco (Optional[float]): Preço máximo do produto para filtrar. Se None, não filtra por preço máximo.
        skip (int): Número de produtos a pular para paginação. Padrão é 0.
        limit (int): Número máximo de produtos a retornar. Padrão é 100.

    Returns:
        List[Produto]: Lista de produtos que atendem aos critérios de

## 4. Gerar README.md

Criar um README completo para o projeto.


In [7]:
# Informações do projeto para gerar README
project_info = {
    "name": "Produtos API",
    "description": "API REST para gerenciamento de produtos de um e-commerce",
    "language": "Python",
    "framework": "FastAPI",
    "features": [
        "CRUD completo de produtos",
        "Filtros por categoria, preço e status",
        "Busca por texto",
        "Paginação",
        "Validação automática com Pydantic"
    ],
    "dependencies": [
        "fastapi>=0.111.0",
        "uvicorn>=0.30.0",
        "pydantic>=2.8.0"
    ],
    "author": "Seu Nome"
}

print("📋 Informações do projeto:")
for key, value in project_info.items():
    print(f"  {key}: {value}")


📋 Informações do projeto:
  name: Produtos API
  description: API REST para gerenciamento de produtos de um e-commerce
  language: Python
  framework: FastAPI
  features: ['CRUD completo de produtos', 'Filtros por categoria, preço e status', 'Busca por texto', 'Paginação', 'Validação automática com Pydantic']
  dependencies: ['fastapi>=0.111.0', 'uvicorn>=0.30.0', 'pydantic>=2.8.0']
  author: Seu Nome


In [8]:
# Gerar README
print("📋 Gerando README.md...\n")

resultado = doc_copilot.generate_readme(project_info)

if resultado.success:
    print("✅ README gerado:\n")
    print(resultado.content)
else:
    print(f"❌ Erro: {resultado.content}")


📋 Gerando README.md...

✅ README gerado:

```markdown
# Produtos API

![Build Status](https://img.shields.io/badge/build-passing-brightgreen)
![Version](https://img.shields.io/badge/version-1.0.0-blue)
![License](https://img.shields.io/badge/license-MIT-green)

## Descrição do Projeto

Produtos API é uma API REST desenvolvida em Python para o gerenciamento de produtos em um e-commerce. A API oferece funcionalidades completas de CRUD (Create, Read, Update, Delete) para produtos, além de filtros avançados, busca por texto, paginação e validação automática utilizando Pydantic.

## Features/Funcionalidades

- **CRUD Completo de Produtos:** Criação, leitura, atualização e exclusão de produtos.
- **Filtros Avançados:** Filtragem de produtos por categoria, preço e status.
- **Busca por Texto:** Pesquisa de produtos com base em palavras-chave.
- **Paginação:** Suporte a paginação para facilitar a navegação por grandes volumes de dados.
- **Validação Automática:** Uso do Pydantic para validação

## 5. Adicionar Comentários Inline

Adicionar comentários explicativos em código complexo.


In [9]:
# Código complexo sem comentários
codigo_complexo = '''
def merge_sort(arr):
    if len(arr) <= 1:
        return arr
    
    mid = len(arr) // 2
    left = merge_sort(arr[:mid])
    right = merge_sort(arr[mid:])
    
    return merge(left, right)

def merge(left, right):
    result = []
    i = j = 0
    
    while i < len(left) and j < len(right):
        if left[i] <= right[j]:
            result.append(left[i])
            i += 1
        else:
            result.append(right[j])
            j += 1
    
    result.extend(left[i:])
    result.extend(right[j:])
    
    return result

def binary_search(arr, target, low=0, high=None):
    if high is None:
        high = len(arr) - 1
    
    if low > high:
        return -1
    
    mid = (low + high) // 2
    
    if arr[mid] == target:
        return mid
    elif arr[mid] > target:
        return binary_search(arr, target, low, mid - 1)
    else:
        return binary_search(arr, target, mid + 1, high)
'''

print("📝 Código complexo sem comentários:")
print(codigo_complexo)


📝 Código complexo sem comentários:

def merge_sort(arr):
    if len(arr) <= 1:
        return arr

    mid = len(arr) // 2
    left = merge_sort(arr[:mid])
    right = merge_sort(arr[mid:])

    return merge(left, right)

def merge(left, right):
    result = []
    i = j = 0

    while i < len(left) and j < len(right):
        if left[i] <= right[j]:
            result.append(left[i])
            i += 1
        else:
            result.append(right[j])
            j += 1

    result.extend(left[i:])
    result.extend(right[j:])

    return result

def binary_search(arr, target, low=0, high=None):
    if high is None:
        high = len(arr) - 1

    if low > high:
        return -1

    mid = (low + high) // 2

    if arr[mid] == target:
        return mid
    elif arr[mid] > target:
        return binary_search(arr, target, low, mid - 1)
    else:
        return binary_search(arr, target, mid + 1, high)



In [10]:
# Adicionar comentários inline
print("💬 Adicionando comentários explicativos...\n")

resultado = doc_copilot.add_inline_comments(
    code=codigo_complexo,
    language="python"
)

if resultado.success:
    print("✅ Código comentado:\n")
    print(resultado.content)
else:
    print(f"❌ Erro: {resultado.content}")


💬 Adicionando comentários explicativos...

✅ Código comentado:

```python
def merge_sort(arr):
    # Base case: if the array has 1 or no elements, it's already sorted
    if len(arr) <= 1:
        return arr

    # Find the middle index to split the array into two halves
    mid = len(arr) // 2

    # Recursively sort the left half
    left = merge_sort(arr[:mid])

    # Recursively sort the right half
    right = merge_sort(arr[mid:])

    # Merge the sorted halves and return
    return merge(left, right)

def merge(left, right):
    result = []  # This will store the merged sorted array
    i = j = 0  # Pointers for left and right arrays

    # Compare elements from both arrays and append the smaller one
    while i < len(left) and j < len(right):
        if left[i] <= right[j]:
            result.append(left[i])
            i += 1
        else:
            result.append(right[j])
            j += 1

    # Append any remaining elements from the left array
    result.extend(left[i:])


## 6. Resumo

O **Documentation Copilot** oferece:

| Método | Descrição |
|--------|-----------|
| `generate_docstring()` | Gera docstrings para funções/classes |
| `generate_readme()` | Cria README.md completo |
| `add_inline_comments()` | Adiciona comentários explicativos |
| `document_file()` | Documenta arquivo completo |

### Estilos de Docstring Suportados

| Estilo | Exemplo |
|--------|---------|
| **Google** | `Args:`, `Returns:`, `Raises:` |
| **NumPy** | `Parameters`, `Returns`, `Examples` |
| **Sphinx** | `:param:`, `:returns:`, `:raises:` |

### Dicas para Boa Documentação

1. **Seja claro e conciso** - Evite jargões desnecessários
2. **Inclua exemplos** - Código de exemplo ajuda muito
3. **Documente exceções** - Liste os erros que podem ocorrer
4. **Mantenha atualizado** - Documentação desatualizada é pior que nenhuma


In [11]:
print("🎉 Demo concluída!")
print("\n📚 Próximos passos:")
print("  1. Use generate_docstring() para documentar suas funções")
print("  2. Gere READMEs para seus projetos")
print("  3. Adicione comentários em código complexo")
print("  4. Experimente diferentes estilos (google, numpy, sphinx)")


🎉 Demo concluída!

📚 Próximos passos:
  1. Use generate_docstring() para documentar suas funções
  2. Gere READMEs para seus projetos
  3. Adicione comentários em código complexo
  4. Experimente diferentes estilos (google, numpy, sphinx)
